# Day3_0 별첨: 다중 상속 (Multiple Inheritance) - 정답

이 노트북은 Day3_0 별첨 다중 상속 실습 퀴즈의 정답 풀이를 제공합니다.

---

## Q1. 기본 다중 상속 ⭐

**문제**: `Robot` 클래스를 만드세요.

**요구사항**:
- `Movable` 클래스: `move()` 메서드 ("이동합니다" 반환)
- `Speakable` 클래스: `speak()` 메서드 ("말합니다" 반환)
- `Robot` 클래스: 위 두 클래스를 상속하고, `name` 속성 추가
- 모든 메서드는 `self.name`을 포함한 메시지 반환

In [ ]:
# 정답
class Movable:
    """이동 능력"""
    def move(self):
        return f"{self.name}가 이동합니다"

class Speakable:
    """말하기 능력"""
    def speak(self):
        return f"{self.name}가 말합니다"

class Robot(Movable, Speakable):
    """로봇 (이동 + 말하기)"""
    def __init__(self, name):
        self.name = name

# 테스트
robot = Robot("R2D2")
print(robot.move())
print(robot.speak())

### 💡 해설

- **다중 상속 문법**: `class Robot(Movable, Speakable)`
- **self.name 접근**: 부모 클래스 메서드에서 자식의 속성 접근 가능
- **순서**: `Movable`을 먼저 상속하므로 MRO에서도 먼저 탐색

---

## Q2. MRO 이해하기 ⭐⭐

**문제**: 다음 클래스 계층에서 MRO를 예측하고 확인하세요.

In [ ]:
# 정답
class Base:
    def method(self):
        return "Base"

class Left(Base):
    def method(self):
        return "Left"

class Right(Base):
    def method(self):
        return "Right"

class Child(Left, Right):
    pass

# 테스트
child = Child()
print("=== MRO ===")
print(f"MRO: {[cls.__name__ for cls in Child.__mro__]}")
print()

# 자세히
for i, cls in enumerate(Child.__mro__):
    print(f"{i+1}. {cls.__name__}")

print(f"\nchild.method() = {child.method()}")
print("→ Child → Left → Right → Base 순서로 탐색")
print("→ Left에서 method()를 찾았으므로 'Left' 반환")

### 💡 해설

**MRO 순서**: Child → Left → Right → Base → object

- **1. Child**: 자식 클래스 우선
- **2. Left**: 첫 번째 부모 (왼쪽부터)
- **3. Right**: 두 번째 부모
- **4. Base**: 공통 부모 (Left, Right 이후)
- **5. object**: 최상위 클래스

**C3 Linearization** 알고리즘:
- 자식 → 부모 (왼쪽부터)
- 공통 조상은 마지막에 한 번만

**메서드 탐색**:
1. Child에서 `method()` 탐색 → 없음
2. Left에서 `method()` 탐색 → **찾음!** → `"Left"` 반환

---

## Q3. super()로 다이아몬드 해결 ⭐⭐⭐

**문제**: 다이아몬드 상속 구조에서 `super()`를 올바르게 사용하세요.

In [ ]:
# 정답
class Vehicle:
    def __init__(self, brand):
        print(f"Vehicle.__init__({brand})")
        self.brand = brand

class Car(Vehicle):
    def __init__(self, brand, model):
        print(f"Car.__init__({brand}, {model})")
        super().__init__(brand)  # Vehicle.__init__ 호출
        self.model = model

class Electric:
    def __init__(self, battery_capacity, **kwargs):
        print(f"Electric.__init__({battery_capacity})")
        # **kwargs를 super()에 전달 (다중 상속 시 중요!)
        super().__init__(**kwargs)
        self.battery_capacity = battery_capacity

class ElectricCar(Car, Electric):
    def __init__(self, brand, model, battery_capacity):
        print(f"ElectricCar.__init__({brand}, {model}, {battery_capacity})")
        # super()로 MRO 순서대로 호출
        super().__init__(brand=brand, model=model, battery_capacity=battery_capacity)

# 테스트
print("=== ElectricCar 생성 ===")
tesla = ElectricCar("Tesla", "Model 3", 75)

print("\n=== 속성 확인 ===")
print(f"브랜드: {tesla.brand}")
print(f"모델: {tesla.model}")
print(f"배터리: {tesla.battery_capacity}kWh")

print("\n=== MRO ===")
for i, cls in enumerate(ElectricCar.__mro__):
    print(f"{i+1}. {cls.__name__}")

### 💡 해설

**핵심 포인트**:

1. **Electric 클래스의 `**kwargs`**:
   ```python
   def __init__(self, battery_capacity, **kwargs):
       super().__init__(**kwargs)  # 나머지 인자 전달
   ```
   - MRO에서 Electric 다음이 Vehicle이므로, `brand`를 Vehicle로 전달해야 함
   - `**kwargs`로 받아서 `super()`로 전달

2. **ElectricCar의 키워드 인자**:
   ```python
   super().__init__(brand=brand, model=model, battery_capacity=battery_capacity)
   ```
   - 키워드 인자로 전달하면 MRO 순서대로 각 클래스가 필요한 인자를 가져감

**호출 순서** (MRO: ElectricCar → Car → Electric → Vehicle):
1. `ElectricCar.__init__` 호출
2. → `Car.__init__` 호출 (model 처리)
3. → `Electric.__init__` 호출 (battery_capacity 처리)
4. → `Vehicle.__init__` 호출 (brand 처리)

**Vehicle.__init__은 단 한 번만 호출됩니다!**

---

## Q4. Mixin 패턴 구현 ⭐⭐⭐⭐

**문제**: 로깅 기능을 제공하는 Mixin을 만드세요.

In [ ]:
# 정답
class LoggingMixin:
    """로깅 기능 제공 Mixin"""
    
    def log(self, level, message):
        """기본 로그 메서드"""
        print(f"[{level}] {message}")
    
    def info(self, message):
        """INFO 레벨 로그"""
        self.log("INFO", message)
    
    def warning(self, message):
        """WARNING 레벨 로그"""
        self.log("WARNING", message)
    
    def error(self, message):
        """ERROR 레벨 로그"""
        self.log("ERROR", message)

class User:
    """기본 사용자 클래스"""
    def __init__(self, name, email):
        self.name = name
        self.email = email
    
    def login(self):
        return f"{self.name} 로그인"
    
    def logout(self):
        return f"{self.name} 로그아웃"

class AdminUser(LoggingMixin, User):
    """관리자 사용자 (로깅 기능 포함)"""
    
    def login(self):
        result = super().login()
        self.info(f"{self.name}님이 로그인했습니다.")
        return result
    
    def logout(self):
        result = super().logout()
        self.info(f"{self.name}님이 로그아웃했습니다.")
        return result
    
    def delete_user(self, username):
        """사용자 삭제 (관리자 전용)"""
        self.warning(f"관리자 {self.name}님이 사용자 '{username}'를 삭제했습니다.")
        return f"사용자 '{username}' 삭제 완료"

# 테스트
admin = AdminUser("admin", "admin@example.com")
admin.login()
admin.delete_user("old_user")
admin.logout()

### 💡 해설

**LoggingMixin 특징**:
- `__init__` 없음 → 독립적
- 단일 기능 (로깅)만 제공
- 다른 클래스와 조합하여 사용

**AdminUser 구조**:
```python
class AdminUser(LoggingMixin, User):
```
- LoggingMixin으로부터 로깅 메서드 상속
- User로부터 기본 사용자 기능 상속
- MRO: AdminUser → LoggingMixin → User

**Mixin 활용 패턴**:
1. `login()` 오버라이딩 → `super().login()` 호출 → `info()` 로그
2. `delete_user()` 신규 메서드 → `warning()` 로그

**실무 확장**:
```python
# 다양한 Mixin 조합
class SuperAdminUser(LoggingMixin, TimestampMixin, ValidationMixin, User):
    # 로깅 + 타임스탬프 + 검증 + 사용자 기능
    pass
```

---

## Q5. 종합 실무 예제 ⭐⭐⭐⭐⭐

**문제**: 데이터베이스 모델을 위한 Mixin 시스템을 만드세요.

In [ ]:
# 정답
from datetime import datetime
import time

class TimestampMixin:
    """생성/수정 시각 자동 관리"""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.created_at = datetime.now()
        self.updated_at = datetime.now()
    
    def touch(self):
        """수정 시각 업데이트"""
        self.updated_at = datetime.now()

class ValidationMixin:
    """검증 기능 제공"""
    def validate(self):
        """추상 메서드 - 서브클래스에서 구현"""
        raise NotImplementedError("서브클래스에서 validate()를 구현해야 합니다.")
    
    def is_valid(self):
        """검증 실행 및 결과 반환"""
        try:
            self.validate()
            return True
        except ValueError as e:
            print(f"검증 실패: {e}")
            return False

class SerializableMixin:
    """직렬화 기능 제공"""
    def to_dict(self):
        """모든 속성을 딕셔너리로 변환"""
        result = {}
        for key, value in self.__dict__.items():
            if isinstance(value, datetime):
                # datetime을 ISO 형식 문자열로 변환
                result[key] = value.isoformat()
            else:
                result[key] = value
        return result

class Product(TimestampMixin, ValidationMixin, SerializableMixin):
    """제품 모델 (3개 Mixin 활용)"""
    def __init__(self, name, price, stock):
        self.name = name
        self.price = price
        self.stock = stock
        # TimestampMixin의 __init__ 호출
        super().__init__()
    
    def validate(self):
        """검증 로직 구현"""
        if self.price <= 0:
            raise ValueError(f"가격은 0보다 커야 합니다: {self.price}")
        if self.stock < 0:
            raise ValueError(f"재고는 0 이상이어야 합니다: {self.stock}")
    
    def sell(self, quantity):
        """판매 처리"""
        if quantity > self.stock:
            return f"재고 부족! 현재 재고: {self.stock}개"
        
        self.stock -= quantity
        self.touch()  # TimestampMixin
        return f"{self.name} {quantity}개 판매 완료. 남은 재고: {self.stock}개"

# 테스트
product = Product("노트북", 1500000, 10)

print("=== 유효성 검사 ===")
print(f"유효성 검사: {product.is_valid()}")

print("\n=== 초기 데이터 ===")
import json
print(json.dumps(product.to_dict(), ensure_ascii=False, indent=2))

print("\n=== 판매 처리 ===")
time.sleep(1)  # 시간 차이 만들기
result = product.sell(3)
print(result)

print("\n=== 판매 후 데이터 ===")
print(json.dumps(product.to_dict(), ensure_ascii=False, indent=2))

print("\n=== 잘못된 제품 ===")
bad_product = Product("에러상품", -1000, -5)
print(f"유효성 검사: {bad_product.is_valid()}")

### 💡 해설

**1. TimestampMixin**:
```python
def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)  # 다음 클래스로 전달
    self.created_at = datetime.now()
    self.updated_at = datetime.now()
```
- `*args, **kwargs`로 인자 받아 `super()`로 전달
- 다중 상속에서 필수 패턴!

**2. ValidationMixin**:
```python
def validate(self):
    raise NotImplementedError  # 추상 메서드 패턴

def is_valid(self):
    try:
        self.validate()  # 서브클래스 구현 호출
        return True
    except ValueError:
        return False
```
- Template Method 패턴
- 서브클래스에서 `validate()` 구현 강제

**3. SerializableMixin**:
```python
def to_dict(self):
    result = {}
    for key, value in self.__dict__.items():
        if isinstance(value, datetime):
            result[key] = value.isoformat()  # datetime 변환
        else:
            result[key] = value
    return result
```
- 모든 속성을 순회하며 딕셔너리 생성
- datetime 타입만 특별 처리

**4. Product 클래스**:
```python
class Product(TimestampMixin, ValidationMixin, SerializableMixin):
    def __init__(self, name, price, stock):
        self.name = name
        self.price = price
        self.stock = stock
        super().__init__()  # TimestampMixin 초기화
```

**MRO**: Product → TimestampMixin → ValidationMixin → SerializableMixin → object

**실무 활용**:
- Django Model Mixin
- SQLAlchemy Mixin
- FastAPI Pydantic Model

---

## 🎯 핵심 패턴 정리

### 1. Mixin `__init__` 패턴

```python
class MyMixin:
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)  # 필수!
        # Mixin 초기화
```

**이유**: 다중 상속 시 MRO를 따라 올바르게 초기화되도록

### 2. 추상 메서드 패턴

```python
class ValidationMixin:
    def validate(self):
        raise NotImplementedError  # 서브클래스에서 구현 강제
    
    def is_valid(self):
        try:
            self.validate()  # 서브클래스 호출
            return True
        except:
            return False
```

### 3. 키워드 인자 전달 패턴

```python
class Parent1:
    def __init__(self, a, **kwargs):
        super().__init__(**kwargs)
        self.a = a

class Parent2:
    def __init__(self, b, **kwargs):
        super().__init__(**kwargs)
        self.b = b

class Child(Parent1, Parent2):
    def __init__(self, a, b, c):
        super().__init__(a=a, b=b)  # 키워드 인자로 전달
        self.c = c
```

### 4. MRO 확인 패턴

```python
# 클래스 MRO 확인
print(MyClass.__mro__)

# 읽기 쉽게
for cls in MyClass.__mro__:
    print(cls.__name__)
```

---

## 📚 실무 Best Practices

### ✅ DO (권장)

1. **Mixin 명명**: `XxxMixin` 접미사 사용
2. **단일 책임**: 하나의 Mixin은 하나의 기능만
3. **super() 사용**: 항상 `super()`로 부모 호출
4. **문서화**: Mixin의 역할과 사용법 명확히 기술
5. **순서 주의**: Mixin을 왼쪽, 기본 클래스를 오른쪽에 배치

### ❌ DON'T (비권장)

1. **직접 호출**: `Parent.__init__(self)` 대신 `super()` 사용
2. **복잡한 다이아몬드**: 가능한 단순한 구조 유지
3. **Mixin 간 의존성**: Mixin끼리 의존하지 않도록
4. **과도한 Mixin**: 너무 많은 Mixin 조합은 복잡도 증가
5. **`__init__` 생략**: Mixin에서 `super().__init__()` 생략 금지

---

## 🔍 디버깅 팁

### MRO 문제 해결

```python
# 1. MRO 확인
print(MyClass.__mro__)

# 2. 메서드 탐색 경로 확인
print(MyClass.method)  # bound method 정보

# 3. 속성 위치 확인
import inspect
print(inspect.getmro(MyClass))
```

### 초기화 순서 디버깅

```python
class DebugMixin:
    def __init__(self, *args, **kwargs):
        print(f"{self.__class__.__name__}.__init__ 호출")
        print(f"  args: {args}")
        print(f"  kwargs: {kwargs}")
        super().__init__(*args, **kwargs)
```

**수고하셨습니다!** 🎉